# Task 4


## Analysis of data


In this notebook, we are going to analyse the data from the mnist dataset. We will also train a model, make predictions and visualize the predictions after the fact. 


### Importing packages and modules

In [ ]:
import 